## Loading the Data

In [1]:
from datasets import load_from_disk

dataset = load_from_disk('ds')

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['bo', 'en', 'source_file'],
        num_rows: 122058
    })
    test: Dataset({
        features: ['bo', 'en', 'source_file'],
        num_rows: 13562
    })
})

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('billingsmoore/getok-v0')
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-efficient-tiny", device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

## Preprocess Data

The dataset can now be tokenized for training.

In [4]:
def bo_en_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['bo']]
    translation_targets = [example for example in examples['en']]
    
    # Tokenize translation inputs and targets
    bo_en_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return bo_en_model_inputs

In [5]:
tokenized_dataset = dataset.map(bo_en_preprocess_function, batched=True)

Map:   0%|          | 0/122058 [00:00<?, ? examples/s]

Map:   0%|          | 0/13562 [00:00<?, ? examples/s]

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [6]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [7]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute TER score
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score = ter_result["score"]

    # Return rounded results
    metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

    #print("Computed Metrics:", metrics)

    return metrics

In [8]:
import wandb

wandb.login()

wandb: Currently logged in as: billingsmoore to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"bidirectional",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_5215/4140109040.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/45774 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'loss': 1.3098, 'grad_norm': 0.11990063637495041, 'learning_rate': 0.00029672303054135535, 'epoch': 0.03}
{'loss': 0.7439, 'grad_norm': 0.10276002436876297, 'learning_rate': 0.0002934460610827107, 'epoch': 0.07}
{'loss': 0.6824, 'grad_norm': 0.07932163029909134, 'learning_rate': 0.00029016909162406603, 'epoch': 0.1}
{'loss': 0.666, 'grad_norm': 0.0764455795288086, 'learning_rate': 0.0002868921221654214, 'epoch': 0.13}
{'loss': 0.6616, 'grad_norm': 0.09991148114204407, 'learning_rate': 0.0002836151527067767, 'epoch': 0.16}
{'loss': 0.6315, 'grad_norm': 0.29187387228012085, 'learning_rate': 0.0002803381832481321, 'epoch': 0.2}
{'loss': 0.6206, 'grad_norm': 0.16956818103790283, 'learning_rate': 0.00027706121378948747, 'epoch': 0.23}
{'loss': 0.6217, 'grad_norm': 0.14320552349090576, 'learning_rate': 0.00027378424433084284, 'epoch': 0.26}
{'loss': 0.62, 'grad_norm': 0.05835013836622238, 'learning_rate': 0.00027050727487219815, 'epoch': 0.29}
{'loss': 0.5842, 'grad_norm': 0.214022248983383

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1696 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.3972046375274658, 'eval_bleu': 8.1267, 'eval_chrf': 18.7476, 'eval_ter': 97.4764, 'eval_runtime': 155.6257, 'eval_samples_per_second': 87.145, 'eval_steps_per_second': 10.898, 'epoch': 3.0}
{'train_runtime': 6536.421, 'train_samples_per_second': 56.021, 'train_steps_per_second': 7.003, 'train_loss': 0.4981749754576642, 'epoch': 3.0}


TrainOutput(global_step=45774, training_loss=0.4981749754576642, metrics={'train_runtime': 6536.421, 'train_samples_per_second': 56.021, 'train_steps_per_second': 7.003, 'total_flos': 4841624105164800.0, 'train_loss': 0.4981749754576642, 'epoch': 3.0})